In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 3, ndecoderlayers = 0, dropout = 0.3):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(3, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        #mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        #mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1, corner_points = None, initial_triangulation = None):
        """
        corner_points, etc., shoudn't include a batch dimension
        """
        if corner_points == None:
            ncornerpoints = 4
        else:
            ncornerpoints = corner_points.size(0)
        if npoints <= ncornerpoints:
            print("Error: not enough points for valid problem instance")
            return
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints - ncornerpoints, 3], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints - ncornerpoints, 3)
        )
        if corner_points == None:
            self.corner_points = torch.tensor(
                [[0, 0, 0], [3, 0, 0], [0, 3, 0], [0, 0, 3]], dtype = floattype, device=device
            )
        else:
            self.corner_points = corner_points
        self.points = torch.cat(
            [
                self.corner_points.unsqueeze(0).expand(self.batchsize, -1, -1),
                self.points,
            ],
            dim=-2,
        )  # [batchsize * nsamples, npoints, 3]
        self.points_mask = torch.cat(
            [
                torch.ones([self.batchsize, ncornerpoints], dtype=torch.bool, device=device),
                torch.zeros(
                    [self.batchsize, npoints - ncornerpoints], dtype=torch.bool, device=device
                ),
            ],
            dim=1,
        )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )

        """
        points are now triples
        triangles are now quadruples
        edges are now still just indices, but there are four of them per 'triangle', and they correspond to triples of points, not pairs
        we use  0,2,1  0,3,2  0,1,3  1,2,3  as the order of the four 'edges'/faces
        opposite face is always ordered such that the last two indices are swapped
        faces are always read ANTICLOCKWISE
        
        first three points of tetrahedron MUST be read clockwise (from the outside) to get correct sign on incircle test
        
        new point will be inserted in zeroth position, so if corresponding face of REMOVED tetrahedron is [x,y,z] (being read anticlockwise from outside in) new tetrahedron is [p, x, y, z]
        """
        
        """
        number of tetrahedra is not the same for each batch (in 3D), so store as a big list, and remember batch index that it comes from
        """
        if corner_points == None:
            initial_triangulation = torch.tensor([[0, 1, 2, 3]], dtype=torch.long, device=device)
        
        self.partial_delaunay_triangles = initial_triangulation.unsqueeze(0).expand(self.batchsize, -1, -1).reshape(-1, 4)
        self.batch_index = torch.arange(self.batchsize, dtype = torch.long, device = device).unsqueeze(1).expand(-1, initial_triangulation.size(0)).reshape(-1)
        
        self.batch_triangles = self.partial_delaunay_triangles.size(0) #[0]
        self.ntriangles = torch.full([self.batchsize], initial_triangulation.size(0), dtype = torch.long, device = device) #[self.batchsize]
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert point_index.size(0) == self.batchsize
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        triangles_coordinates = self.points[self.batch_index.unsqueeze(1), self.partial_delaunay_triangles] # [batch_triangles, 4, 3]
        
        newpoint = self.points[self.batch_index, point_index[self.batch_index]] # [batch_triangles, 3]

        incircle_matrix = torch.cat(
            [
                newpoint.unsqueeze(1),
                triangles_coordinates,
            ],
            dim=-2,
        )  # [batch_triangles, 5, 3]
        incircle_matrix = torch.cat(
            [
                (incircle_matrix * incircle_matrix).sum(-1, keepdim=True),
                incircle_matrix,
                torch.ones([self.batch_triangles, 5, 1], dtype = floattype, device=device),
            ],
            dim=-1,
        )  # [batch_triangles, 5, 5]
        assert incircle_matrix.dtype == floattype
        assert str(incircle_matrix.device) == device
        
        incircle_test = (
            incircle_matrix.det() > 0
        )  # [batch_triangles], is True if inside incircle
        
        conflicts = incircle_test.sum()
        
        conflicting_triangles = self.partial_delaunay_triangles[incircle_test] # [conflicts, 4]
        
        conflicting_edges_index0 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([0, 0, 0, 1])
        conflicting_edges_index0 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index1 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([2, 3, 1, 2])
        conflicting_edges_index1 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges_index2 = torch.empty_like(conflicting_triangles)
        indices = torch.LongTensor([1, 2, 3, 3])
        conflicting_edges_index2 = conflicting_triangles[:, indices] # [conflicts, 4]
        
        conflicting_edges = torch.cat([conflicting_edges_index0.view(-1).unsqueeze(-1), conflicting_edges_index1.view(-1).unsqueeze(-1), conflicting_edges_index2.view(-1).unsqueeze(-1)], dim = -1).reshape(-1, 3) # [conflicts * 4, 3]
        
        edge_batch_index = self.batch_index[incircle_test].unsqueeze(1).expand(-1, 4).reshape(-1) # [conflicts * 4]
        
        indices = torch.LongTensor([0, 2, 1])
        comparison_edges = conflicting_edges[:, indices] # [conflicts * 4, 3]        
        
        unravel_nomatch_mask = torch.ones([conflicts * 4], dtype = torch.bool, device = device) # [conflicts * 4]
        i = 1
        while True:
            
            todo_mask = unravel_nomatch_mask[:-i].logical_and(edge_batch_index[:-i] == edge_batch_index[i:])
            if i % 4 == 0:
                if todo_mask.sum() == 0:
                    break
            
            match_mask = todo_mask.clone()
            match_mask[todo_mask] = (conflicting_edges[:-i][todo_mask] != comparison_edges[i:][todo_mask]).sum(-1).logical_not()
            
            unravel_nomatch_mask[:-i][match_mask] = False
            unravel_nomatch_mask[i:][match_mask] = False
            
            i += 1
        
        batch_newtriangles = unravel_nomatch_mask.sum()
        
        nomatch_edges = conflicting_edges[unravel_nomatch_mask] # [batch_newtriangles, 3], already in correct order to insert into 1,2,3 (since already anticlockwise from outside in)
        assert list(nomatch_edges.size()) == [batch_newtriangles, 3]
        nomatch_batch_index = edge_batch_index[unravel_nomatch_mask] # [batch_newtriangles]
        
        nomatch_newpoint = point_index[nomatch_batch_index] # [batch_newtriangles]
        
        newtriangles = torch.cat([nomatch_newpoint.unsqueeze(1), nomatch_edges], dim = -1) # [batch_newtriangles, 4]
        
        
        nremoved_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        nnew_triangles = torch.zeros([self.batchsize], dtype = torch.long, device = device)
        
        indices = self.batch_index[incircle_test]
        nremoved_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        indices = edge_batch_index[unravel_nomatch_mask]
        nnew_triangles.put_(indices, torch.ones_like(indices, dtype = torch.long), accumulate = True) # [batchsize]
        
        assert (nnew_triangles <= 2 * nremoved_triangles + 2).logical_not().sum().logical_not()
        
        """
        NOTE:
        I THINK it's possible for nnew_triangles to be less than nremoved_triangles (or my code is just buggy...)
        """
        
        assert nnew_triangles.sum() == batch_newtriangles
        assert nremoved_triangles.sum() == incircle_test.sum()
        
        nadditional_triangles = nnew_triangles - nremoved_triangles # [batchsize]
        ntriangles = self.ntriangles + nadditional_triangles # [batchsize]
        
        partial_delaunay_triangles = torch.empty([ntriangles.sum(), 4], dtype = torch.long, device = device)
        batch_index = torch.empty([ntriangles.sum()], dtype = torch.long, device = device)
        
        cumulative_triangles = torch.cat([torch.zeros([1], dtype = torch.long, device = device), nnew_triangles.cumsum(0)[:-1]]) # [batchsize], cumulative sum starts at zero
        
        """
        since may actually have LESS triangles than previous round, we insert all that survive into the first slots (in that batch)
        """
        good_triangle_indices = torch.arange(incircle_test.logical_not().sum(), dtype = torch.long, device = device)
        good_triangle_indices += cumulative_triangles[self.batch_index[incircle_test.logical_not()]]
        bad_triangle_indices_mask = torch.ones([ntriangles.sum(0)], dtype = torch.bool, device = device)
        bad_triangle_indices_mask.scatter_(0, good_triangle_indices, False)
        
        assert good_triangle_indices.size(0) == incircle_test.logical_not().sum()
        assert bad_triangle_indices_mask.sum() == batch_newtriangles
        
        partial_delaunay_triangles[good_triangle_indices] = self.partial_delaunay_triangles[~incircle_test]
        batch_index[good_triangle_indices] = self.batch_index[~incircle_test]
        
        partial_delaunay_triangles[bad_triangle_indices_mask] = newtriangles
        batch_index[bad_triangle_indices_mask] = nomatch_batch_index
        
        self.partial_delaunay_triangles = partial_delaunay_triangles
        self.batch_index = batch_index
        
        self.ntriangles = ntriangles
        self.batch_triangles = self.partial_delaunay_triangles.size(0)
        
        self.points_mask.scatter_(
            1, point_index.unsqueeze(1).expand(-1, self.npoints), True
        )
        self.points_sequence = torch.cat(
            [self.points_sequence, point_index.unsqueeze(1)], dim=1
        )
        
        self.cost += nremoved_triangles
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    

env = environment()


def train(epochs = 30000, npoints = 14, batchsize = 100, nsamples = 8):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(4, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > 0.3
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        loss = ( 
                #- logprob.view(batchsize, nsamples)[:, :-1] 
                #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
                #* positive_reinforcement_binary 
                logprob.view(batchsize, nsamples)[:, :-1] 
                * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 5) 
                * negative_reinforcement_binary 
        ).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
        

In [ ]:
train(npoints = 14, batchsize = 2000, nsamples = 8) #greedyprob - 5, without +ve reinforcement, 
#consider negative if performance > 0.3 + greedy

49.43700408935547 -13.896759033203125 -15.042184829711914 -13.664446830749512 -15.079228401184082 -15.079229354858398
48.28750228881836 -13.529680252075195 -14.96078872680664 -13.570863723754883 -14.974246978759766 -14.97424602508545
47.867000579833984 -13.08080768585205 -14.838079452514648 -13.421530723571777 -14.518027305603027 -14.518028259277344
47.35300064086914 -12.740721702575684 -14.722257614135742 -13.38096809387207 -15.293867111206055 -15.293869018554688
46.91100311279297 -12.191612243652344 -14.501062393188477 -12.380035400390625 -14.571846961975098 -14.571847915649414
46.720001220703125 -11.722293853759766 -14.238434791564941 -10.819051742553711 -15.369571685791016 -15.3695707321167
46.863502502441406 -12.347016334533691 -14.566247940063477 -11.810909271240234 -15.741226196289062 -15.741226196289062
46.795501708984375 -12.623443603515625 -14.698280334472656 -11.94289779663086 -15.787503242492676 -15.787503242492676
46.77850341796875 -12.343954086303711 -14.568873405456543 -

45.816001892089844 -11.641711235046387 -14.288346290588379 -12.51633071899414 -13.957342147827148 -13.957341194152832
45.44600296020508 -11.630784034729004 -14.299622535705566 -11.717619895935059 -13.607528686523438 -13.607527732849121
45.394500732421875 -11.63644027709961 -14.300901412963867 -11.255569458007812 -12.943599700927734 -12.943599700927734
45.14900207519531 -11.649333000183105 -14.323932647705078 -11.902397155761719 -15.728522300720215 -15.728522300720215
45.47300338745117 -11.775840759277344 -14.410319328308105 -12.17764663696289 -15.25849723815918 -15.258498191833496
45.208003997802734 -11.923855781555176 -14.493654251098633 -12.18389892578125 -12.461669921875 -12.461669921875
45.32100296020508 -11.979573249816895 -14.541921615600586 -12.014209747314453 -14.341390609741211 -14.341390609741211
45.56850051879883 -11.8853759765625 -14.49885368347168 -10.87310791015625 -14.2520170211792 -14.2520170211792
45.51400375366211 -11.69393253326416 -14.391151428222656 -12.08907699584

44.81500244140625 -11.599690437316895 -14.337368965148926 -12.409914016723633 -14.40066909790039 -14.40066909790039
44.82650375366211 -11.504196166992188 -14.223870277404785 -11.087818145751953 -14.83576774597168 -14.83576774597168
44.82500076293945 -11.578008651733398 -14.231196403503418 -11.679523468017578 -15.279212951660156 -15.279212951660156
44.76000213623047 -11.670427322387695 -14.306829452514648 -11.260787963867188 -13.770634651184082 -13.770633697509766
44.54750061035156 -11.655088424682617 -14.3091402053833 -12.125280380249023 -15.676708221435547 -15.676708221435547
44.606502532958984 -11.55713939666748 -14.26256275177002 -11.030111312866211 -11.843660354614258 -11.843659400939941
44.432003021240234 -11.443221092224121 -14.201539993286133 -11.65031623840332 -11.776007652282715 -11.776006698608398
44.48600387573242 -11.40878963470459 -14.223855018615723 -11.408491134643555 -16.458044052124023 -16.458044052124023
44.600502014160156 -11.409831047058105 -14.261086463928223 -11.0

43.84050369262695 -11.583965301513672 -14.366281509399414 -11.701051712036133 -12.440370559692383 -12.4403715133667
43.9630012512207 -11.508995056152344 -14.264893531799316 -11.076655387878418 -12.845230102539062 -12.845229148864746
43.893001556396484 -11.440190315246582 -14.176521301269531 -11.07948112487793 -14.243432998657227 -14.24343204498291
43.8025016784668 -11.451512336730957 -14.201691627502441 -11.983555793762207 -11.456047058105469 -11.456048011779785
43.66550064086914 -11.534289360046387 -14.254963874816895 -11.772439956665039 -13.207215309143066 -13.207215309143066
43.71000289916992 -11.653990745544434 -14.333003044128418 -11.962589263916016 -13.167887687683105 -13.167888641357422
43.85000228881836 -11.666303634643555 -14.373371124267578 -11.680546760559082 -15.840498924255371 -15.840497970581055
43.76700210571289 -11.680063247680664 -14.37761116027832 -12.098088264465332 -14.901113510131836 -14.90111255645752
43.78350067138672 -11.588396072387695 -14.336274147033691 -11.8

43.426002502441406 -11.433374404907227 -14.25390625 -11.272918701171875 -13.381989479064941 -13.381988525390625
43.49400329589844 -11.481064796447754 -14.305424690246582 -11.026081085205078 -14.113981246948242 -14.113981246948242
43.40650177001953 -11.552722930908203 -14.322482109069824 -11.187175750732422 -14.929465293884277 -14.92946720123291
43.457000732421875 -11.600242614746094 -14.353874206542969 -11.815367698669434 -12.505126953125 -12.505126953125
43.40300369262695 -11.604461669921875 -14.377341270446777 -11.735849380493164 -13.248939514160156 -13.248940467834473
43.154502868652344 -11.487122535705566 -14.2842435836792 -11.929413795471191 -14.907485008239746 -14.907485008239746
43.35350036621094 -11.42954158782959 -14.274707794189453 -11.238709449768066 -14.671710014343262 -14.671710014343262
43.48250198364258 -11.419868469238281 -14.266138076782227 -11.137006759643555 -13.665223121643066 -13.66522216796875
43.176002502441406 -11.445549011230469 -14.319564819335938 -11.22580909

...

41.59450149536133 -11.489395141601562 -14.534415245056152 -12.099040985107422 -14.43567943572998 -14.435680389404297
41.39500045776367 -11.34903335571289 -14.463541984558105 -10.76510238647461 -15.701513290405273 -15.701515197753906
41.381500244140625 -11.307538032531738 -14.436182975769043 -9.960986137390137 -13.63048267364502 -13.63048267364502
41.38100051879883 -11.31396198272705 -14.416383743286133 -10.835477828979492 -15.503554344177246 -15.503552436828613
41.4995002746582 -11.339713096618652 -14.442350387573242 -11.099205017089844 -14.935579299926758 -14.935578346252441
41.40250015258789 -11.35077953338623 -14.442276954650879 -11.547569274902344 -12.951679229736328 -12.951680183410645
41.369503021240234 -11.417078018188477 -14.48995590209961 -11.082859992980957 -14.713501930236816 -14.713501930236816
41.54650115966797 -11.433188438415527 -14.50806713104248 -11.014246940612793 -14.685619354248047 -14.685619354248047
41.569000244140625 -11.448355674743652 -14.54086971282959 -11.208

41.558502197265625 -11.393559455871582 -14.508475303649902 -10.88355827331543 -16.138208389282227 -16.138208389282227
41.38350296020508 -11.338363647460938 -14.488780975341797 -11.060127258300781 -15.686861991882324 -15.68686294555664
41.51100158691406 -11.299176216125488 -14.486801147460938 -10.426867485046387 -15.076691627502441 -15.076690673828125
41.427001953125 -11.33479118347168 -14.48652172088623 -12.593986511230469 -12.721593856811523 -12.721593856811523
41.41550064086914 -11.37462329864502 -14.517048835754395 -11.05157470703125 -14.922394752502441 -14.922394752502441
41.729000091552734 -11.414994239807129 -14.495718955993652 -11.230903625488281 -15.483299255371094 -15.483297348022461
41.682003021240234 -11.43384075164795 -14.469865798950195 -11.378290176391602 -12.990596771240234 -12.990596771240234
41.60300064086914 -11.489700317382812 -14.511555671691895 -11.535099029541016 -12.70680046081543 -12.706801414489746
41.702003479003906 -11.528437614440918 -14.502191543579102 -10.

41.415000915527344 -11.334125518798828 -14.441551208496094 -13.049888610839844 -13.355327606201172 -13.355327606201172
41.20800018310547 -11.35437297821045 -14.457539558410645 -11.595748901367188 -15.799310684204102 -15.799311637878418
41.44600296020508 -11.433090209960938 -14.500104904174805 -10.97496223449707 -14.343896865844727 -14.34389591217041
41.61600112915039 -11.495375633239746 -14.51297378540039 -11.21243667602539 -15.140183448791504 -15.140183448791504
41.45600128173828 -11.471559524536133 -14.531734466552734 -10.477673530578613 -14.549514770507812 -14.54951286315918
41.393001556396484 -11.397638320922852 -14.488990783691406 -11.662681579589844 -14.910888671875 -14.910888671875
41.352500915527344 -11.33055305480957 -14.46364974975586 -10.82655143737793 -12.063446044921875 -12.063445091247559
41.28700256347656 -11.301363945007324 -14.39885139465332 -10.918272018432617 -14.884611129760742 -14.884613037109375
41.37750244140625 -11.309672355651855 -14.398719787597656 -11.2674865

41.406002044677734 -11.370349884033203 -14.422223091125488 -11.221368789672852 -13.865346908569336 -13.865345001220703
41.51400375366211 -11.380697250366211 -14.446744918823242 -11.066654205322266 -13.794870376586914 -13.794870376586914
41.34400177001953 -11.399454116821289 -14.48046875 -10.99299144744873 -13.45419692993164 -13.454195022583008
41.45750045776367 -11.42794132232666 -14.486913681030273 -12.284645080566406 -13.975391387939453 -13.97539234161377
41.432003021240234 -11.389430046081543 -14.502923965454102 -11.037445068359375 -16.883079528808594 -16.88307762145996
41.696502685546875 -11.399188041687012 -14.488140106201172 -11.930227279663086 -14.83456039428711 -14.83456039428711
41.5150032043457 -11.398539543151855 -14.523246765136719 -10.897988319396973 -14.64260482788086 -14.642603874206543
41.506500244140625 -11.488616943359375 -14.54499340057373 -11.449563980102539 -16.78711700439453 -16.78711700439453
41.41400146484375 -11.492445945739746 -14.547083854675293 -10.518803596

41.307003021240234 -11.310445785522461 -14.4796781539917 -11.975008010864258 -13.021059036254883 -13.021059036254883
41.54100036621094 -11.333930969238281 -14.485410690307617 -11.72169303894043 -14.677873611450195 -14.677872657775879
41.16600036621094 -11.376964569091797 -14.51141357421875 -11.616504669189453 -15.928362846374512 -15.928360939025879
41.352500915527344 -11.446913719177246 -14.506490707397461 -12.783300399780273 -12.109878540039062 -12.109878540039062
41.358001708984375 -11.486714363098145 -14.507822036743164 -10.848756790161133 -15.811216354370117 -15.811217308044434
41.299503326416016 -11.523319244384766 -14.508541107177734 -11.398811340332031 -15.162209510803223 -15.162208557128906
41.172000885009766 -11.51538372039795 -14.481755256652832 -11.016597747802734 -15.542032241821289 -15.542032241821289
41.43400192260742 -11.51883602142334 -14.464606285095215 -11.352810859680176 -15.537591934204102 -15.537593841552734
41.17400360107422 -11.44222354888916 -14.432316780090332 